In [7]:
import sys
import os

# Add the project root to the path
sys.path.append(os.path.abspath(os.path.join('..')))



from src.extract.google_trends import fetch_trends_data
from src.config.keywords import FINTECH_KEYWORDS
from src.config.countries import COUNTRIES 

In [8]:
df_test = fetch_trends_data("billetera digital","PE")
df_test.head()


,date,keyword,country_code,interest_score
0,2019-01-01,billetera digital,PE,0
1,2019-02-01,billetera digital,PE,0
2,2019-03-01,billetera digital,PE,0
3,2019-04-01,billetera digital,PE,0
4,2019-05-01,billetera digital,PE,0


In [13]:
df_test.describe()


,date,interest_score
count,73,73.000000
mean,2021-12-31 04:16:26,3.342466
min,2019-01-01 00:00:00,0.000000
25%,2020-07-01 00:00:00,0.000000
50%,2022-01-01 00:00:00,1.000000
75%,2023-07-01 00:00:00,2.000000
max,2025-01-01 00:00:00,100.000000
std,NaN,12.087895


In [17]:
df_test["interest_score"].max()

np.int64(100)

In [18]:
df_test = fetch_trends_data("Yape", "PE")
df_test.head()
df_test.describe()


,date,interest_score
count,73,73.000000
mean,2021-12-31 04:16:26,46.410959
min,2019-01-01 00:00:00,3.000000
25%,2020-07-01 00:00:00,25.000000
50%,2022-01-01 00:00:00,45.000000
75%,2023-07-01 00:00:00,73.000000
max,2025-01-01 00:00:00,100.000000
std,NaN,29.375838


In [4]:
import sys
import os

sys.path.append(os.path.abspath(".."))


from src.transform.clean_transform import clean_trends_dataframe

df_raw = fetch_trends_data("Yape", "PE")
df_clean = clean_trends_dataframe(df_raw)

df_clean.info()


<class 'pandas.DataFrame'>
RangeIndex: 73 entries, 0 to 72
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype        
---  ------          --------------  -----        
 0   date            73 non-null     datetime64[s]
 1   keyword         73 non-null     str          
 2   country_code    73 non-null     str          
 3   interest_score  73 non-null     int64        
dtypes: datetime64[s](1), int64(1), str(2)
memory usage: 2.4 KB


In [9]:
import pandas as pd
import time

from src.extract.google_trends import fetch_trends_data
from src.transform.clean_transform import clean_trends_dataframe
from src.config.keywords import FINTECH_KEYWORDS
from src.config.countries import COUNTRIES


In [ ]:

all_data = []

for country_code in COUNTRIES.keys():
    for keyword in FINTECH_KEYWORDS:
        try:
            print(f"Fetching: {keyword} - {country_code}")
            
            df_raw = fetch_trends_data(keyword, country_code)
            
            if df_raw is None or df_raw.empty:
                print(f" Empty data for {keyword} - {country_code}")
                continue
            
            df_clean = clean_trends_dataframe(df_raw)
            all_data.append(df_clean)
            
            time.sleep(2)
             
        except Exception as e:
            print(f" Error with {keyword} - {country_code}: {e}")
            continue

print("Total DataFrames collected:", len(all_data))

if len(all_data) > 0:
    df_master = pd.concat(all_data, ignore_index=True)
    df_master.info()
else:
    print("No data collected. Check extraction function.")


Fetching: billetera digital - PE
Fetching: Yape - PE
Fetching: Plin - PE
Fetching: fintech - PE
Fetching: neobanco - PE
Fetching: criptomonedas - PE
Fetching: pagos digitales - PE
Fetching: tarjeta virtual - PE
Fetching: transferencias digitales - PE
⚠️ Empty data for transferencias digitales - PE
Fetching: billetera digital - MX
Fetching: Yape - MX
Fetching: Plin - MX
Fetching: fintech - MX
Fetching: neobanco - MX
Fetching: criptomonedas - MX
Fetching: pagos digitales - MX
Fetching: tarjeta virtual - MX
Fetching: transferencias digitales - MX
Fetching: billetera digital - CO
Fetching: Yape - CO
Fetching: Plin - CO
Fetching: fintech - CO
Fetching: neobanco - CO
Fetching: criptomonedas - CO
Fetching: pagos digitales - CO
Fetching: tarjeta virtual - CO
Fetching: transferencias digitales - CO
⚠️ Empty data for transferencias digitales - CO
Fetching: billetera digital - CL
❌ Error with billetera digital - CL: The request failed: Google returned a response with code 429
Fetching: Yape - CL


In [11]:
df_master.groupby(["country_code", "keyword"]).size()


country_code  keyword                 
CO            Plin                        73
              Yape                        52
              billetera digital           48
              criptomonedas               73
              fintech                     73
              neobanco                    21
              pagos digitales             34
              tarjeta virtual             73
MX            Plin                        73
              Yape                        69
              billetera digital           15
              criptomonedas               73
              fintech                     73
              neobanco                    17
              pagos digitales             73
              tarjeta virtual             73
              transferencias digitales     3
PE            Plin                        73
              Yape                        73
              billetera digital           51
              criptomonedas               73
              fi

In [12]:
df_master.isnull().sum()

date              0
keyword           0
country_code      0
interest_score    0
dtype: int64

In [ ]:
df_master["interest_score"].describe()

count    1362.000000
mean       48.527900
std        25.644226
min         1.000000
25%        27.000000
50%        49.000000
75%        70.000000
max       100.000000
Name: interest_score, dtype: float64

In [14]:
df_master.groupby("country_code")["interest_score"].mean().sort_values(ascending=False)

country_code
MX    51.957356
CO    50.246085
PE    43.199552
Name: interest_score, dtype: float64

In [ ]:
df_master.groupby("keyword")["interest_score"].mean().sort_values(ascending=False)

keyword
transferencias digitales    78.333333
fintech                     65.954338
tarjeta virtual             64.292237
neobanco                    62.738095
pagos digitales             52.962406
Plin                        42.643836
Yape                        40.221649
criptomonedas               35.077626
billetera digital           24.850877
Name: interest_score, dtype: float64

In [16]:
df_master.groupby(["country_code","keyword"])["interest_score"].mean().sort_values(ascending=False)


country_code  keyword                 
MX            transferencias digitales    78.333333
CO            tarjeta virtual             72.123288
PE            fintech                     70.890411
MX            fintech                     70.506849
              tarjeta virtual             69.904110
              billetera digital           69.200000
CO            neobanco                    68.714286
              pagos digitales             65.823529
MX            neobanco                    61.647059
CO            fintech                     56.465753
PE            tarjeta virtual             50.849315
CO            Plin                        50.712329
MX            pagos digitales             49.164384
PE            Yape                        46.945205
              pagos digitales             46.807692
MX            Plin                        40.958904
              criptomonedas               37.109589
PE            criptomonedas               37.027397
MX            Yape       

In [20]:
# Graficos
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(12,6))
sns.lineplot(data=df_master, x="date", y="interest_score", hue="keyword")
plt.show()


ModuleNotFoundError: No module named 'seaborn'

In [ ]:
+